In [1]:
#no-spark option for parsing data by size and truncating long values and removing unnecessary sensors
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pickle as pkl
from datetime import datetime
from numpy import int64
import sys
import csv
import os
#WSL links
#AOT_ROOT='/mnt/e/datasets/Chicago-AoT-dataset/AoT_Chicago.complete.2022-08-31/'
#OUT_ROOT='/mnt/e/outputs/chicago_aot/'
#Linux Links
AOT_ROOT='/media/kyleddm/Lab SSD 2/Chicago-AoT-dataset/AoT_Chicago.complete.2022-08-31/'
OUT_ROOT='/media/kyleddm/Lab SSD 2/outputs/chicago_aot/'
max_id_substring='001e061' #this is the portion of the node_id that is identical to every node
strip_sens=['chemsense', 'metsense', 'loadavg', 'mem', 'time', 'device', 'net_rx', 'net_tx', 'ping', 'media', 'modem', 'disk_used', 'disk_size', 'disk_used_ratio', 'service_active', 'plugins', 'wagman_fc', 'wagman_cu', 'wagman_enabled', 'wagman_vdc', 'wagman_hb', 'wagman_stopping', 'wagman_starting', 'wagman_killing', 'wagman_th', 'wagman_comm', 'wagman_uptime', 'image_detector']
subsystems={'chemsense':'cs', 'metsense':'ms', 'lightsense':'ls', 'alphasense':'as', 'plantower':'pt', 'audio':'aud', 'wagman':'wg', 'microphone':'mic', 'image':'img'}
trunc_sens={'pressure':'pres','temperature':'temp','humidity':'hum'}
def truncateSubsystem(df):
    return subsystems[df['subsystem']] if df['subsystem'] in subsystems.keys() else df['subsystem']
def truncateID(df):
    return df['node_id'].replace(max_id_substring,'')
def truncateSensor(df):
    return trunc_sens[df['parameter']] if df['parameter'] in trunc_sens.keys() else df['parameter']
def calcUnixTime(timestamp:str):
    from datetime import datetime
    import pytz
    #code from Bing 2025OCT116
    # Define the date, time, and timezone
    date_str = timestamp#"2025-10-16 15:30:00"  # Example date and time
    timezone_str = "UTC"#"America/New_York"  # Example timezone
    # Parse the date and time
    dt = datetime.strptime(date_str, "%Y/%m/%d %H:%M:%S")
    # Localize the datetime object to the specified timezone
    timezone = pytz.timezone(timezone_str)
    localized_dt = timezone.localize(dt)
    # Convert to UNIX timestamp
    unix_timestamp = int(localized_dt.timestamp())
    print("UNIX Timestamp:", unix_timestamp)
    return unix_timestamp
def mergeFiles(fileList):
    #all_files = glob.glob(os.path.join(path, "*.csv"))
    mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)
    return mergedDF

In [ ]:
#this converts the size of the subsuystem parameter to be smaller to take up less space
#test_chunk=pd.read_csv(AOT_ROOT+'data.csv', chunksize=1)
#chunk=next(test_chunk)
files=os.listdir(OUT_ROOT+'by_date/') #lets us skip writing files we already wrote
subData=[]
chunkSiz=1000000
chunk_count=0
for chunk in pd.read_csv(AOT_ROOT+'data.csv', chunksize=chunkSiz):
    print('Chunk Start Date: '+chunk.iloc[0]['timestamp'])
    chunk['subsystem']=chunk.apply(lambda x: truncateSubsystem(x),axis=1)
    #print(sys.getsizeof(datum))
    chunk['node_id']=chunk.apply(lambda x: truncateID(x),axis=1)
    chunk['parameter']=chunk.apply(lambda x: truncateSensor(x),axis=1)
    print('completed truncation for chunk '+str(chunk_count))
    chunk_count=chunk_count+1
    for i in range(0,chunkSiz):
        datum=chunk.iloc[i].to_dict()
    #print(datum.loc[0])
    #print(datum.loc[0])
    #print(sys.getsizeof(datum))
        if str(datum['sensor']) in strip_sens:
            pass
        else:
            subData.append(datum)
        if sys.getsizeof(subData)> 50000000: #50,000,000 bytes -> 50MB (base 10)
            title=subData[-1]['timestamp'].replace('/','-').replace(' ','_').replace(':','.')
            print(title)
            if title+'.csv' in files:
                print('skipped '+title)
            else:
                df=pd.DataFrame(subData)
                df.to_csv(OUT_ROOT+title+'.csv',index=False)
                #myFile = open(OUT_ROOT+title+'.csv', 'w')
                #writer = csv.writer(myFile)
                #writer.writerow(subData[0].keys())
                #for dictionary in subData:
                #    writer.writerow(dictionary.values())
                #myFile.close()
                df=None
            subData=[]
#the last chunk will error out.  This allows manual saving.
title=subData[-1]['timestamp'].replace('/','-').replace(' ','_').replace(':','.')
df=pd.DataFrame(subData)
df.to_csv(OUT_ROOT+title+'.csv',index=False)

In [ ]:
#This group of cells analyzes the dataset to determine what we can prune.  This is not necessary to run
#this will fail on opcN2 because its a collection of bins skip if can't be a number.  I'm saving it off as a string for now.
#it looks as though there's reports for most sensors in the real-valued column so we can read in each 300MB file, strip the raw column, eliminate any row with nans in the real-value column, then save it off again.  This should allow more data in fewer files.
data_location=OUT_ROOT+'by_date/'
data_file='2019-12-01_11.05.35.csv'#'2018-01-09_10.48.06.csv'
df=pd.read_csv(data_location+data_file)
sencols=df['sensor'].unique()
print(sencols)
subcols=df['subsystem'].unique()
print(subcols)
paramcols=df['parameter'].unique()
print(paramcols)
stats_list=[]
for sub in subcols:
    for sen in sencols:
        for param in paramcols:
            filtered_df = df[df['subsystem'] == sub]
            filtered_df = filtered_df[filtered_df['sensor'] == sen]
            filtered_df = filtered_df[filtered_df['parameter'] == param]
            if not filtered_df.empty:
                stats={}
                stats['id']=sub+'_'+sen+'_'+param
                try:
                    stats['minVal']=min(filtered_df['value_hrf'].astype('float', errors='ignore'))
                    stats['maxVal']=max(filtered_df['value_hrf'].astype('float', errors='ignore'))
                    stats['meanVal']=np.mean(filtered_df['value_hrf'].astype('float', errors='ignore'))
                    stats['stdVal']=np.std(filtered_df['value_hrf'].astype('float', errors='ignore'))
                    stats['minVal_raw']=min(filtered_df['value_raw'].astype('float', errors='ignore'))
                    stats['maxVal_raw']=max(filtered_df['value_raw'].astype('float', errors='ignore'))
                    stats['meanVal_raw']=np.mean(filtered_df['value_raw'].astype('float', errors='ignore'))
                    stats['stdVal_raw']=np.std(filtered_df['value_raw'].astype('float', errors='ignore'))
                except Exception as e:
                    print(f"An error occurred: {e}")
                    stats['minVal']=filtered_df['value_hrf'].iloc[0]
                    stats['maxVal']='NA'
                    stats['meanVal']='NA'
                    stats['stdVal']='NA'
                    stats['minVal_raw']=filtered_df['value_raw'].iloc[0]
                    stats['maxVal_raw']='NA'
                    stats['meanVal_raw']='NA'
                    stats['stdVal_raw']='NA'
                stats_list.append(stats)
                
                

In [ ]:
#This cell shrinks the small files into smaller files by pruning unnecessary data
#We're going to eliminate the value_raw from each 300MB file, then check, row by row, if each value_hrf is empty.  If so, we strip the row.
files=os.listdir(OUT_ROOT)
for i in range(0,len(files)):
    print(files[i])
    test_file=OUT_ROOT+files[i]
    testDF=pd.read_csv(test_file,low_memory=False)
    print(testDF.shape)
    testDF=testDF.drop('value_raw',axis=1)
    testDF=testDF.dropna(subset=['value_hrf'])
    print(testDF.shape)
    testDF.to_csv(OUT_ROOT+'by_date_pruned/'+files[i], index=False)


In [2]:
#this cell takes the smaller files and merges them into larger files
newFileLoc=OUT_ROOT+'by_date_pruned/'
merge_dir=os.listdir(newFileLoc)
#print(OUT_ROOT+'by_date/'+merge_dir[0])
#print(os.path.getsize(OUT_ROOT+'by_date/'+merge_dir[0]))#bytes to kibibytes
merge_dir.sort()
if 'data_processing_log.txt' in merge_dir:
    merge_dir.remove('data_processing_log.txt')
maxSize=10000000000 #10 GB
totFileSize=0
bigFileList=[]
#gotta sort the file list!  otherwise the elements of the csv files are put into the main file out of order and sorting based on a date string is annoying!
for file in merge_dir:
    currentFileSize=os.path.getsize(newFileLoc+file)
    if totFileSize+currentFileSize<maxSize:
        bigFileList.append(newFileLoc+file)
        totFileSize=totFileSize+currentFileSize
    else:
        bigDF=mergeFiles(bigFileList)
        bigDF.to_csv(OUT_ROOT+'by_date_large/big'+os.path.basename(bigFileList[-1]))
        print(OUT_ROOT+'by_date_large/big'+os.path.basename(bigFileList[-1])+' written.')
        bigDF=None
        bigFileList=[]
        totFileSize=0
        bigFileList.append(newFileLoc+file)
        totFileSize=totFileSize+currentFileSize
        

/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)


/media/kyleddm/Lab SSD 2/outputs/chicago_aot/by_date_large/big2018-03-30_00.36.46.csv written.


/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)


/media/kyleddm/Lab SSD 2/outputs/chicago_aot/by_date_large/big2018-04-29_11.51.23.csv written.


/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)
/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)


...

/media/kyleddm/Lab SSD 2/outputs/chicago_aot/by_date_large/big2019-11-05_12.56.24.csv written.


/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)
/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)
/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)
/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)
/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  m

/media/kyleddm/Lab SSD 2/outputs/chicago_aot/by_date_large/big2020-01-28_03.55.55.csv written.


/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)
/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)
/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)
/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)
/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  m

/media/kyleddm/Lab SSD 2/outputs/chicago_aot/by_date_large/big2020-06-14_00.05.58.csv written.


In [3]:
bigDF=mergeFiles(bigFileList)
bigDF.to_csv(OUT_ROOT+'by_date_large/big'+os.path.basename(bigFileList[-1]))
print(OUT_ROOT+'by_date_large/big'+os.path.basename(bigFileList[-1])+' written.')

/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)
/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)
/tmp/ipykernel_7019/930517811.py:46: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  mergedDF = pd.concat((pd.read_csv(f) for f in fileList), ignore_index=True)


/media/kyleddm/Lab SSD 2/outputs/chicago_aot/by_date_large/big2021-03-22_05.18.49.csv written.
